In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
## --- Author: Ishita Jalan ---

In [ ]:
# Function to convert time series data for each location point to subbasin level data - averaged for each subbasin

def temptosubbasins(directory,tmax,tmin,subbasins,dataname):
    
    os.chdir(directory)
    
    subbasins = pd.read_excel(subbasins, sheet_name = 'Sheet1', engine = 'openpyxl')
    tmax = pd.read_excel(tmax, sheet_name = 'Sheet1', engine = 'openpyxl')
    tmin = pd.read_excel(tmin, sheet_name = 'Sheet1', engine = 'openpyxl')
    
    subbasins.sort_values(by='Subbasin', inplace=True)
    ls_subsn = subbasins['Subbasin'].unique()
    
    final_tmax = pd.DataFrame()
    final_tmin = pd.DataFrame()

    for i in range(len(ls_subsn)):
        print("Processing subbasin number:", ls_subsn[i])
        sb_df = subbasins[subbasins['Subbasin']==ls_subsn[i]]
        ls_loc = sb_df['Location']
    
        tmin_subset = tmin[tmin['Location'].isin(ls_loc)]
        tmin_df = pd.merge(tmin_subset, sb_df, on = 'Location')
        tmin_df.drop(['OBJECTID', 'Latitude', 'Longitude', 'Subbasin', 'gridcode'], axis= 1, inplace = True) #adding the area ocolumn to the tmin data df
        
        tmax_subset = tmax[tmax['Location'].isin(ls_loc)]
        tmax_df = pd.merge(tmax_subset, sb_df, on = 'Location')
        tmax_df.drop(['OBJECTID', 'Latitude', 'Longitude', 'Subbasin', 'gridcode'], axis= 1, inplace = True) #adding the area ocolumn to the tmax data df
    
        sum_area = tmax_df['Cell_Area'].sum() #calculating the total area under the current subbasin
    
        temp_xlen = len(tmax_df.columns)-2 #determining the number of columns in the dataframe
    
        ls_tmax = []
        ls_tmin = []
    
        for j in range(temp_xlen):
            tmin_df[j] = tmin_df[j] * tmin_df['Cell_Area']
            min_avg = tmin_df[j].sum()/sum_area #calculating weighted average for each day of the time series
            ls_tmin.append(min_avg)
        
            tmax_df[j] = tmax_df[j] * tmax_df['Cell_Area']
            max_avg = tmax_df[j].sum()/sum_area
            ls_tmax.append(max_avg)
    
        final_tmin[i] = ls_tmin
        final_tmax[i] = ls_tmax
    
    final_tmin.to_csv(dataname + '_tmin_subbasins.csv')
    final_tmax.to_csv(dataname + '_tmax_subbasins.csv')

In [ ]:
#Calling the function for EWEMBI data
temptosubbasins(r'D:\TUM\Master_Thesis\Benin\Data\EWEMBI_v1.1-second_download\time_series', 'EWEMBI_max_1998-2012.xlsx',
                'EWEMBI_min_1998-2012.xlsx', 'EWE_intersect_subbasins.xlsx','EWEMBI')

In [ ]:
#Calling the function for ERA5 data
temptosubbasins(r'D:\TUM\Master_Thesis\Benin\Data\ERA5\time_series', 'ERA5_max_1998-2012.xlsx', 'ERA5_min_1998-2012.xlsx',
                'ERA5_intersect_subbasins.xlsx', 'ERA5')

In [ ]:
#Calling the function for CFSR data - also eliminate 'Shape_Area' in the function above
temptosubbasins(r'D:\TUM\Master_Thesis\Benin\Data\CFSR', 'CFSR_tmax.xlsx', 'CFSR_tmin.xlsx',
                'CFSR_intersect_subbasins.xlsx', 'CFSR')

In [ ]:
#Calling the function for CPC data
temptosubbasins(r'D:\TUM\Master_Thesis\Benin\Data\CPC\time_series', 'CPC_tmax_1998-2012.xlsx', 'CPC_tmin_1998-2012.xlsx',
                'CPC_intersect_subbasins.xlsx','CPC')

In [ ]:
#Calling the function for PGF data - also change 'Shape_Area' to 'Area' in the function above
temptosubbasins(r'D:\TUM\Master_Thesis\Benin\Data\PGF\time_series', 'PGF_tmax_1998-2012.xlsx', 'PGF_tmin_1998-2012.xlsx',
                'PGF_intersect_subbasins.xlsx','PGF')